### Full POP model works

In [2]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using LinearAlgebra
using Dates
using HDF5
using TSSOS

Consider simple Lindblad master equation with just one dissipator:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\gamma\left[J \rho J^\dagger - \frac{1}{2}\left\{ J^\dagger J, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} e_1 & h_1 - i h_2 \\ h_1 + i h_2 & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J = \begin{pmatrix} a_{11} + i b_{11} & a_{12} \\ a_{21} + i b_{21} & -a_{11} - i b_{11} 
   \end{pmatrix} 
$

or with some additional assumptions

$
J = \begin{pmatrix} a_{11} & a_{12} + i b_{12}  \\ a_{21} + i b_{21} & -a_{11}
   \end{pmatrix} 
$

or assuming zero temperature 

$
J = \begin{pmatrix} a_{11} + i b_{11} & a_{12}   \\ 0 & -a_{11} - i b_{11}
   \end{pmatrix} 
$

In [3]:
@polyvar e[1]
@polyvar h[1:2]


Hˢʸᵐᵇ = [ e[1]               h[1] - im*h[2]
          h[1] + im*h[2]     0.             ]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 e₁                  h₁ + (0.0-1.0im)h₂
 h₁ + (0.0+1.0im)h₂  0.0+0.0im

In [4]:
method = "fix-Phi-up_"

@polyvar a[1:4]
@polyvar b[1:4]


Jˢʸᵐᵇ = [ a[1]+im*b[1]      a[2]
          a[3] + im*b[3]   -a[1]-im*b[1] ]


2×2 Matrix{Polynomial{true, Complex{Int64}}}:
 a₁ + (0+1im)b₁  a₂
 a₃ + (0+1im)b₃  -a₁ + (0-1im)b₁

In [6]:
data_dir = "../DATA/"
println(data_dir)

models_dir = "../MODELS/"
tests_dir = "../TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

../DATA/


In [7]:
all_files = vcat(dodeca_files, basis_files)
train_files = basis_files 
test_files = dodeca_files;

### Train models loop

In [69]:
function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end

function tssos_scaled_min(p)
    
    ################################################################################################
    #
    #   Try TSSOS on polynomial with scaled variables
    #
    ################################################################################################

    pd = p #/ maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    # minimize
    # minimizer_scaled_tssos = nothing

    try
        opt,sol,data = tssos_first(p_scaled, variables(p_scaled), QUIET=true, solution=true, newton=true);
        previous_sol = sol

        while ~isnothing(sol)
            previous_sol = sol
            opt,sol,data = tssos_higher!(data; QUIET=true, solution=true);
        end

        global minimizer_scaled_tssos = scale .* previous_sol
    
    catch
        println("Scaled TSSOS failed")
        global minimizer_scaled_tssos = nothing

    end

    best_solution = variables(p_scaled) => minimizer_scaled_tssos

end


function tssos_min(p)
    
    ################################################################################################
    #
    #   Try TSSOS on polynomial with scaled variables
    #
    ################################################################################################

    pd = p #/ maximum(abs.(coefficients(p)))

    # minimize
    # minimizer_scaled_tssos = nothing

    try
        opt,sol,data = tssos_first(pd, variables(pd), QUIET=true, solution=true, newton=true);
        previous_sol = sol

        while ~isnothing(sol)
            previous_sol = sol
            opt,sol,data = tssos_higher!(data; QUIET=true, solution=true);
        end

        global minimizer_tssos = previous_sol
    
    catch
        println("Scaled TSSOS failed")
        global minimizer_scaled_tssos = nothing

    end

    best_solution = variables(p) => minimizer_tssos

end

tssos_min (generic function with 1 method)

In [24]:
import Base.real

function coef_range(obj)
    maximum(abs.(coefficients(obj)))/minimum(abs.(coefficients(obj)))
end

function remove_micro_coefs(p::AbstractPolynomial, cutoff)
    sum(
        
        [abs(coef) > cutoff ? real(coef) * mon : 0 for (coef, mon) in zip(coefficients(p), monomials(p))]
        )
end

remove_micro_coefs (generic function with 1 method)

In [55]:
objₑₓ = 0

for df_trn in train_files # loop over initial states

    ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", "0.079477")
    end_train = length(tᵗʳⁿ)
    if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
    ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
            
    ρᵗʳⁿₒₚₛ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρᵗʳⁿ]
    tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])

    objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, [Jˢʸᵐᵇ])
    
end
coef_range(objₑₓ)

1.5684654085087506e7

In [56]:
using Optim
using Random

lme_minimizer = [25.126, 0., 0., 0., sqrt(0.079477), 0., 0., 0. ]
lme_solution = variables(objₑₓ) => lme_minimizer


PolyVar{true}[e₁, h₁, h₂, a₁, a₂, a₃, b₁, b₃] => [25.126, 0.0, 0.0, 0.0, 0.28191665435018204, 0.0, 0.0, 0.0]

In [57]:
DynamicPolynomials.subs(objₑₓ, lme_solution)

0.0012685537299148564

In [58]:
function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 100

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end

local_rand_min (generic function with 1 method)

In [59]:
solution = local_rand_min(objₑₓ)
DynamicPolynomials.subs(objₑₓ, solution)

0.0005153348994895168

In [70]:
Hˢⁱᵈₗₘₑ = DynamicPolynomials.subs(Hˢʸᵐᵇ, solution)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (25.1261+0.0im)            (3.65351e-5-4.01437e-5im)
 (3.65351e-5+4.01437e-5im)  0.0+0.0im

In [61]:
Jˢⁱᵈₗₘₑ = DynamicPolynomials.subs(Jˢʸᵐᵇ, solution)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-4.09492e-5+3.72248e-6im)  (0.292+0.0im)
 (0.0665766+0.0043999im)     (4.09492e-5-3.72248e-6im)

In [71]:
tssos_scaled_sol = tssos_scaled_min(objₑₓ)


*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.0005153158540288223



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Found a locally optimal solution by Ipopt, giving an upper bound: 0.01175844.
The relative optimality gap is: 1.124312%.
No higher TS step of the TSSOS hierarchy!


PolyVar{true}[e₁, h₁, h₂, a₁, a₂, a₃, b₁, b₃] => [25.12611349833402, 4.8661167760916026e-5, 7.53553003949256e-5, 2.5187618211492216e-10, 2.3634849448780754e-9, 2.3746940597431727e-10, 2.629798205305818e-10, 1.1054283720542595e-10]

In [72]:
DynamicPolynomials.subs(objₑₓ, tssos_scaled_sol)

0.011758438139736427

In [73]:
tssos_scaled_sol

PolyVar{true}[e₁, h₁, h₂, a₁, a₂, a₃, b₁, b₃] => [25.12611349833402, 4.8661167760916026e-5, 7.53553003949256e-5, 2.5187618211492216e-10, 2.3634849448780754e-9, 2.3746940597431727e-10, 2.629798205305818e-10, 1.1054283720542595e-10]

In [74]:
DynamicPolynomials.subs(Hˢʸᵐᵇ, tssos_scaled_sol)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (25.1261+0.0im)            (4.86612e-5-7.53553e-5im)
 (4.86612e-5+7.53553e-5im)  0.0+0.0im

In [75]:
DynamicPolynomials.subs(Jˢʸᵐᵇ, tssos_scaled_sol)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (2.51876e-10+2.6298e-10im)   (2.36348e-9+0.0im)
 (2.37469e-10+1.10543e-10im)  (-2.51876e-10-2.6298e-10im)

In [77]:
tssos_sol = tssos_min(objₑₓ)

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.0005153423032621579
Global optimality certified with relative optimality gap 0.000001%!
No higher TS step of the TSSOS hierarchy!


PolyVar{true}[e₁, h₁, h₂, a₁, a₂, a₃, b₁, b₃] => [25.126144439904422, 3.6534824860130964e-5, 4.0144503699067563e-5, -4.094614009494279e-5, 0.29199997854948706, 0.06657660775593936, 3.7243339675795935e-6, 0.004399905082151034]

In [78]:
DynamicPolynomials.subs(Jˢʸᵐᵇ, tssos_sol)

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-4.09461e-5+3.72433e-6im)  (0.292+0.0im)
 (0.0665766+0.00439991im)    (4.09461e-5-3.72433e-6im)

In [79]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
models_file = "POP_LME_tssos_vs_random_"*method*date_and_time_string * ".h5"
tests_data_file_name = "POP_NoRmLME_trn4_gap_tst20_"*method*date_and_time_string * ".h5"

println(models_file)

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

for γᵢ in γ
    
    println("---------------")
    println("gamma = ", γᵢ)
    
    Hᴸᴹᴱ = [ 25.126        0
             0             0    ]
    γᶠ = parse(Float64, γᵢ)

    Jᴸᴹᴱ = [ 0    √(γᶠ)
             0     0. + 0im  ]

    objₑₓ = 0
    
    objₗₘₑ = 0
    
    for df_trn in train_files # loop over initial states

        ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
        
        #if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
        
        ρᵗʳⁿₒₚₛ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρᵗʳⁿ]
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])
    
        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, [Jˢʸᵐᵇ])
         
        ρˡᵐᵉ = LiPoSID.Lindblad_time_evolution(basis, ρᵗʳⁿ[1], tᵗʳⁿ, Hᴸᴹᴱ, [Jᴸᴹᴱ])
        objₗₘₑ += LiPoSID.simpson_obj(ρˡᵐᵉ, tᵗʳⁿ,  Hˢʸᵐᵇ, [Jˢʸᵐᵇ])
            
        
    end # of files (initial states) loop
            
    coef_minₑₓ = minimum(abs.(coefficients(objₑₓ)))
    coef_maxₑₓ = maximum(abs.(coefficients(objₑₓ)))

    coefs_gapₑₓ = coef_maxₑₓ/coef_minₑₓ 
            
    coef_minₗₘₑ = minimum(abs.(coefficients(objₗₘₑ)))
    coef_maxₗₘₑ = maximum(abs.(coefficients(objₗₘₑ)))
            
    coefs_gapₗₘₑ= coef_maxₗₘₑ/coef_minₗₘₑ 
       
    println("-------------------------------------------")
    println(" POP on ansatz LME simulation")
            
    #objₗₘₑrm = coefs_gapₗₘₑ > 1e6 ? remove_micro_coefs(objₗₘₑ, cutoffₗₘₑ) : objₗₘₑ
    
    solˢⁱᵈₗₘₑtssos  = tssos_min(objₗₘₑrm)
    solˢⁱᵈₗₘₑrnd  = local_rand_min(objₗₘₑrm)
    
    println("Best metod for POP on LME: ", best_methodˢⁱᵈₗₘₑ)
    println("-------------------------------------------")
    
    solₗₘₑ = [e[1], h[1], h[2], a[1], a[2], a[3], b[1], b[3]] => [25.126, 0, 0, 0,  √(γᶠ), 0, 0, 0]
    
    obj_val_lme = convert(Float64, subs(objₗₘₑ, solₗₘₑ))
    
    obj_val_sid_on_lme = convert(Float64, subs(objₗₘₑ, solˢⁱᵈₗₘₑ))
    
    if obj_val_sid_on_lme < obj_val_lme
        println(" simulated LME overfitting")
    end
    
    Hˢⁱᵈₗₘₑtssos = subs(Hˢʸᵐᵇ, solˢⁱᵈₗₘₑtssos)
    Aˢⁱᵈₗₘₑtssos = subs(Jˢʸᵐᵇ, solˢⁱᵈₗₘₑtssos)

    Hˢⁱᵈₗₘₑrnd = subs(Hˢʸᵐᵇ, solˢⁱᵈₗₘₑrnd)
    Aˢⁱᵈₗₘₑrnd = subs(Jˢʸᵐᵇ, solˢⁱᵈₗₘₑrnd)
      
    println(" POP on ansatz EXACT Kurt data ")
                
    solₑₓtssos = tssos_min(objₑₓ)
    solₑₓrnd = local_rand_min(objₑₓ)
   
    obj_val_lme_on_exact = convert(Float64, subs(objₑₓ, solₗₘₑrnd))
    obj_val_sid_on_exact = convert(Float64, subs(objₑₓ, solₑₓrnd))
    
    if obj_val_sid_on_exact < obj_val_lme_on_exact
        println(" Exact data overfitting")
    end

    Hˢⁱᵈₑₓtssos = DynamicPolynomials.subs(Hˢʸᵐᵇ, solₑₓtssos)
    Aˢⁱᵈₑₓtssos = DynamicPolynomials.subs(Aˢʸᵐᵇ, solₑₓtssos)

    Hˢⁱᵈₑₓrnd = DynamicPolynomials.subs(Hˢʸᵐᵇ, solₑₓrnd)
    Aˢⁱᵈₑₓrnd = DynamicPolynomials.subs(Aˢʸᵐᵇ, solₑₓrnd)
    
    H = DenseOperator(basis, Hˢⁱᵈₑₓ)
    A = DenseOperator(basis, Aˢⁱᵈₑₓ)

    
    h5open(models_dir*models_file,"cw") do fid  # read-write, create file if not existing, preserve existing contents

        γ_group = create_group(fid, "gamma_"*γᵢ)
        
        
        γ_group["coef_min_ex"] = coef_minₑₓ
        γ_group["coef_max_ex"] = coef_maxₑₓ
        γ_group["coef_gap_ex"] = coefs_gapₑₓ
                        
        γ_group["coef_min_lme"] = coef_minₗₘₑ
        γ_group["coef_max_lme"] = coef_maxₗₘₑ
        γ_group["coef_gap_lme"] = coefs_gapₗₘₑ       
                        
        γ_group["obj_val_lme"] = obj_val_lme
        γ_group["obj_val_sid_on_lme"] = obj_val_sid_on_lme
        
        γ_group["obj_val_lme_on_exact"] = obj_val_lme_on_exact
        γ_group["obj_val_sid_on_exact"] = obj_val_sid_on_exact


        γ_group["H_sid_sb_tssos"] = convert.(ComplexF64, Hˢⁱᵈₑₓtssos)
        γ_group["A_sid_sb_tssos"] = convert.(ComplexF64, Aˢⁱᵈₑₓtssos)

        γ_group["H_sid_sb_rnd"] = convert.(ComplexF64, Hˢⁱᵈₑₓrnd)
        γ_group["A_sid_sb_rnd"] = convert.(ComplexF64, Aˢⁱᵈₑₓrnd)
        
        γ_group["H_lme"] = convert.(ComplexF64, Hᴸᴹᴱ)
        γ_group["A_lme"] = convert.(ComplexF64, Aᴸᴹᴱ)
        
        γ_group["H_sid_lme_tssos"] = convert.(ComplexF64, Hˢⁱᵈₗₘₑtssos)
        γ_group["A_sid_lme_tssos"] = convert.(ComplexF64, Aˢⁱᵈₗₘₑtssos)

        γ_group["H_sid_lme_rnd"] = convert.(ComplexF64, Hˢⁱᵈₗₘₑrnd)
        γ_group["A_sid_lme_rnd"] = convert.(ComplexF64, Aˢⁱᵈₗₘₑrnd)

        
    end # of HDF5 writing
    
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ)
    end 
    
    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]

        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tᴸᴹᴱ, ρᴸᴹᴱ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hᴸᴹᴱ), [DenseOperator(basis,Aᴸᴹᴱ)])
        tˢⁱᵈₗₘ, ρˢⁱᵈₗₘₑ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₗₘₑ), [DenseOperator(basis,Aˢⁱᵈₗₘₑ)])
        tˢⁱᵈₑₓ, ρˢⁱᵈₑₓ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₑₓ), [DenseOperator(basis,Aˢⁱᵈₑₓ)])   
        

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρᴸᴹᴱ)]
        Fˢⁱᵈᴸᴹᴱₗₘₑ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρˢⁱᵈₗₘₑ, ρᴸᴹᴱ)]
        Fˢⁱᵈᵉˣₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈₑₓ)]
                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["F_lme_exact"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            init_state_group["F_sidlme_lme"] = convert.(Float64, Fˢⁱᵈᴸᴹᴱₗₘₑ)
            init_state_group["F_sidexact_exact"] = convert.(Float64, Fˢⁱᵈᵉˣₑₓ)
            
        end
                    
        print(minimum(Fᴸᴹᴱₑₓ), minimum(Fˢⁱᵈᴸᴹᴱₗₘₑ), minimum(Fˢⁱᵈᵉˣₑₓ))
        println(" ")
    
    end
                
    println()
    println()
                            

end

POP_NoRmLME_trn4_gap_fix-Phi-up_2024-Jan-09_at_13-10.h5
---------------
gamma = 0.079477


UndefVarError: UndefVarError: `end_train` not defined

In [19]:
tests_data_file_name

"POP_NoRmLME_trn4_gap_tst20_fix-Phi-up_2024-Jan-08_at_19-30.h5"